# ACTGAN

This Notebook provides an overview of how to use ACTGAN. It is compatable with SDV CTGAN from version 0.17.X of SDV. The notable changes are exposed through additional keyword parameters when creating the `ACTGAN` instance. Specifically:

- Binary encoding usage. CTGAN uses One Hot Encoding for discrete/categorical columns which can lead to memory issues depending on the cardinality of these columns. You may now specify a cardinality cutoff that will trigger the switch to using a binary encoder, which saves significant memory usage.


- Auto datetime detection. When enabled, each column will be scanned for potential DateTime values. The strfmt of each column will be determined and the underlying SDV Table Metadata will be automatically configured to use a `UnixTimestampEncoder` for these columns. This will give better variability during data sampling and prevent DateTime
columns from being treated as categorical.

- Empty field detection.  Any columns that are empty (or all NaN) will be transformed for fitting and reverse transformed to being empty during sampling. Empty columns can cause training execptions otherwise.


- Epoch callback. Optionally allow the passing of an `EpochInfo` object to any callable when a training epoch completes.

In [ ]:
import pandas as pd

from gretel_synthetics.actgan import ACTGAN

In [ ]:
train_df = pd.read_csv("http://gretel-public-website.s3-website-us-west-2.amazonaws.com/datasets/311_call_center_10k.csv")
train_df.head()


In [ ]:
class EpochTracker:
    """
    Simple example that just accumulates ``EpochInfo`` events,
    but demonstrates how you can route epoch information to
    arbitrary callables during model fitting.
    """
    
    def __init__(self):
        self.epochs = []
        
    def add(self, epoch_data):
        self.epochs.append(epoch_data)
        
epoch_tracker = EpochTracker()

In [ ]:
model = ACTGAN(
    verbose=True,
    binary_encoder_cutoff=10, # use a binary encoder for data transforms if the cardinality of a column is below this value
    auto_transform_datetimes=True,
    epochs=100,
    epoch_callback=epoch_tracker.add
)

In [ ]:
model.fit(train_df)

In [ ]:
# Tracked and stored epoch information

epoch_tracker.epochs[42]

In [ ]:
syn_df = model.sample(100)
syn_df.head()